## Top N Region Analysis

In [1]:
import boto3
import json
from collections import Counter

In [2]:
stream_name = 'order_created'
topn = 10

In [3]:
# end point url is critical for local testing
kinesis_client = boto3.client('kinesis', endpoint_url='http://localhost:4568', region_name='us-east-1'
                                  , aws_access_key_id='x',aws_secret_access_key='x')
# get the shard id
response = kinesis_client.describe_stream(StreamName=stream_name)
shard_id = response['StreamDescription']['Shards'][0]['ShardId']
# get the starting shard iterator. Last Sequence number iterator can also be used to start from the last read record
shard_iterator = kinesis_client.get_shard_iterator(StreamName=stream_name, ShardId=shard_id
                                                   , ShardIteratorType='TRIM_HORIZON')['ShardIterator']
ship_from_region = []

while True:
    out = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=10)
        
    if not out['Records']:
        break;

    for rec in out["Records"]:
        jdat = json.loads(rec["Data"])
        ship_from_region.append(jdat['ship_from_region'])
            
    shard_iterator = out["NextShardIterator"]
    
#print ship_from_region

In [4]:
region_counts = Counter(ship_from_region)

In [5]:
region_counts.most_common(topn)

[(u'reg69', 42),
 (u'reg85', 40),
 (u'reg76', 38),
 (u'reg23', 37),
 (u'reg32', 37),
 (u'reg13', 36),
 (u'reg72', 35),
 (u'reg47', 34),
 (u'reg25', 33),
 (u'reg89', 33)]